In [1]:
import transformers, re, datasets, torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification


dataset = pd.read_csv(r'C:\Users\FLopezP\Desktop\PCIC\Segundo Semestre\Minería de textos\Corpus TASS\corpusTASS-2020\test.tsv', sep = '\t')
dataset.head(5)

,id,texto,pais,etiqueta
0,770809220291760128,Quiero que alguien me diga cosas bonitas,ES,N
1,772825676793589760,@RavlRVara ¡Síguenos porfa! Tenemos que decirt...,ES,NEU
2,772508582923337729,@Charlywayty @Karim0931 cualquiera diría que e...,ES,NEU
3,790657169876193281,"a to esto, el tio de ono no ha venio",ES,N
4,770967255773507584,@IKEASpain es el mismo así que tomo nota de to...,ES,P


In [2]:
def no_arrobas(tweet):
    return re.sub("@+([a-zA-Z|0-9])+", '', tweet)

def create_dataset(texto, tag, clean):
    """
    texto = list ; Conjunto de tweets
    tag = list ; Conjunto de etiquetas
    clean = bool ; Opción de preprocesamiento
    """
    if clean:
        texto = [no_arrobas(_) for _ in texto]
    dict_dataset = {'Tweets': texto, 'Tags': tag}
    aux_df = pd.DataFrame(dict_dataset)
    return Dataset.from_pandas(aux_df)

In [3]:
ds_limpio = create_dataset(dataset['texto'], dataset['etiqueta'], True)
ds_normal = create_dataset(dataset['texto'], dataset['etiqueta'], False)
ds_limpio

Dataset({
    features: ['Tweets', 'Tags'],
    num_rows: 7264
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis")

In [5]:
import evaluate
f1 = evaluate.load('f1')

In [6]:
inputs = tokenizer(ds_limpio['Tweets'][0], return_tensors = 'pt')
with torch.no_grad():
    logits = model(**inputs).logits

pred_class = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]
pred_class

tensor([2])

In [ ]:
# https://huggingface.co/docs/transformers/en/model_doc/bert#transformers.BertForSequenceClassification